---
# WEEK 16

2024/07/22 - 2024/07/28

## 深度学习 C5_W2

### 1. 词语嵌入

- **词语嵌入**：
  - 是一种将词汇表示为低维向量的技术，捕捉词汇之间的语义关系。
  - 常用的词嵌入方法包括Word2Vec、GloVe等。

- **词向量**：
  - 将词汇表示为连续的实数向量。
  - 向量的维度通常较低，以减少计算复杂度。
  - 相似词汇在向量空间中的距离较近。

### 2. 词向量之间的相似度

- **余弦相似度**：
  - 计算两个向量之间的余弦值，范围在-1到1之间。
  - 相似度公式：\[ \text{cosine\_similarity}(A, B) = \frac{A \cdot B}{||A|| \cdot ||B||} \]

- **代码示例**：

```python
import numpy as np

def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)
```

### 3. 使用单词嵌入解决单词类比问题

- **单词类比问题**：
  - 给定词对(A:B)和一个词C，找到D使得C:D与A:B的关系相似。
  - 例如，“男人:女人”与“国王:王后”的类比问题。

- **代码示例**：

```python
def find_analogy(word_a, word_b, word_c, word_vectors):
    vec_a = word_vectors[word_a]
    vec_b = word_vectors[word_b]
    vec_c = word_vectors[word_c]
    
    analogy_vec = vec_b - vec_a + vec_c
    similarities = {word: cosine_similarity(analogy_vec, vec) for word, vec in word_vectors.items()}
    most_similar = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    
    return most_similar[0][0]

# 示例使用预训练的词向量
word_vectors = {'man': np.array([0.2, 0.4]), 'woman': np.array([0.3, 0.5]), 'king': np.array([0.5, 0.7]), 'queen': np.array([0.6, 0.8])}
result = find_analogy('man', 'woman', 'king', word_vectors)
print(result)  # 输出 'queen'
```

### 4. 减少词语嵌入中的偏差

- **偏差问题**：
  - 词向量可能捕捉到数据中的偏见（例如性别、种族等）。
  - 解决方法包括对向量进行去偏处理，确保向量表示的公平性。

### 5. 在 Keras 中使用预训练词向量创建嵌入层

- **嵌入层**：
  - Keras中的嵌入层用于将词汇映射到其对应的词向量。
  - 可以使用预训练的词向量（例如GloVe）来初始化嵌入层。

- **代码示例**：

```python
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
import numpy as np

# 示例预训练词向量（实际应用中应加载真实的预训练词向量）
embedding_matrix = np.array([[0.2, 0.4], [0.3, 0.5], [0.5, 0.7], [0.6, 0.8]])

model = Sequential()
embedding_layer = Embedding(input_dim=4, output_dim=2, weights=[embedding_matrix], trainable=False)
model.add(embedding_layer)

# 示例输入
input_data = np.array([[0, 1, 2], [1, 2, 3]])
output = model.predict(input_data)
print(output)
```

### 6. 负抽样学习词向量

- **负抽样（Negative Sampling）**：
  - 是一种用于训练词向量的技术，通过将预测目标从整个词汇表中的一个词缩小到一个小的负采样集，显著减少计算量。

- **代码示例**：

```python
import numpy as np
import tensorflow as tf

# 假设词汇表和对应的词向量
vocabulary_size = 10000
embedding_dim = 300
embedding_matrix = np.random.rand(vocabulary_size, embedding_dim)

# 创建负采样训练数据
def generate_negative_samples(target_word, context_words, num_negative_samples=5):
    negative_samples = []
    for context_word in context_words:
        negative_samples.extend(np.random.choice(vocabulary_size, num_negative_samples, replace=False))
    return negative_samples

# 负采样训练函数
def train_with_negative_sampling(target_word, context_words, embedding_matrix, learning_rate=0.01):
    target_vector = embedding_matrix[target_word]
    context_vectors = embedding_matrix[context_words]
    negative_samples = generate_negative_samples(target_word, context_words)
    negative_vectors = embedding_matrix[negative_samples]

    positive_loss = -np.log(sigmoid(np.dot(context_vectors, target_vector)))
    negative_loss = -np.log(sigmoid(-np.dot(negative_vectors, target_vector)))

    total_loss = positive_loss + np.sum(negative_loss)
    gradient = total_loss * learning_rate

    embedding_matrix[target_word] -= gradient
    embedding_matrix[context_words] -= gradient
    embedding_matrix[negative_samples] -= gradient

    return embedding_matrix

# Sigmoid函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
```

### 7. GloVe 算法

- **GloVe（Global Vectors for Word Representation）**：
  - 是一种基于词共现矩阵的词向量训练方法。
  - 通过构建词对共现矩阵，优化损失函数来学习词向量。

- **代码示例**：

```python
import numpy as np

def build_cooccurrence_matrix(corpus, vocabulary_size, window_size=5):
    cooccurrence_matrix = np.zeros((vocabulary_size, vocabulary_size))
    for text in corpus:
        words = text.split()
        for i, word in enumerate(words):
            word_index = word_to_index[word]
            context_indices = range(max(0, i - window_size), min(len(words), i + window_size + 1))
            for j in context_indices:
                if i != j:
                    context_word_index = word_to_index[words[j]]
                    cooccurrence_matrix[word_index, context_word_index] += 1
    return cooccurrence_matrix

def train_glove(cooccurrence_matrix, embedding_dim=300, iterations=100, learning_rate=0.01):
    vocabulary_size = cooccurrence_matrix.shape[0]
    W = np.random.rand(vocabulary_size, embedding_dim)
    b = np.random.rand(vocabulary_size)
    for _ in range(iterations):
        for i in range(vocabulary_size):
            for j in range(vocabulary_size):
                if cooccurrence_matrix[i, j] > 0:
                    x_ij = cooccurrence_matrix[i, j]
                    w_i = W[i]
                    w_j = W[j]
                    b_i = b[i]
                    b_j = b[j]
                    f_x = (w_i @ w_j) + b_i + b_j - np.log(x_ij)
                    gradient = f_x * learning_rate
                    W[i] -= gradient
                    W[j] -= gradient
                    b[i] -= gradient
                    b[j] -= gradient
    return W
```

### 8. 词嵌入构建情感分类器

- **情感分类器**：
  - 使用词嵌入作为输入，训练神经网络模型进行文本情感分类。
  - 常用的网络结构包括LSTM、双向LSTM等。

- **代码示例**：

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 示例词嵌入矩阵（实际应用中应加载真实的预训练词向量）
embedding_matrix = np.random.rand(10000, 300)

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=300, weights=[embedding_matrix], trainable=False))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 示例数据
input_data = np.random.randint(10000, size=(1000, 100))
labels = np.random.randint(2, size=(1000, 1))

model.fit(input_data, labels, epochs=10)
```

### 9. LSTM 建立并训练复杂分类器

- **LSTM（长短期记忆网络）**：
  - 是一种改进的RNN，能够更好地捕捉长期依赖关系，缓解梯度消失问题。
  - 适用于处理序列数据，如文本、时间序列等。

- **代码示例**：

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(100, 300)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 示例数据
input_data = np.random.rand(1000, 100, 300)
labels = np.random.randint(2, size=(1000, 1))

model.fit(input_data, labels, epochs=10)
```
